In [1]:
import tkinter as tk
from tkinter import ttk
import socket
import threading
from des_encry_decry import bin_to_hexa,encrypt,pad,decrypt
from rc4 import rc4_encrypt,rc4_decrypt

In [2]:
class CryptoChatServer:
    def __init__(self, root):
        self.root = root
        self.root.title("Server")

        # Create a StringVar to store the selected crypto technique
        self.crypto_technique_var = tk.StringVar()

        # Create a dropdown menu for crypto techniques
        crypto_label = tk.Label(root, text="Crypto Technique:")
        crypto_label.grid(row=0, column=0, padx=10, pady=10, sticky="e")

        crypto_options = ["AES", "RSA", "DES", "RC-4", "ELGAMMAL"]  # Add more as needed
        crypto_dropdown = ttk.Combobox(root, textvariable=self.crypto_technique_var, values=crypto_options)
        crypto_dropdown.grid(row=0, column=1, padx=10, pady=10, sticky="w")
        crypto_dropdown.set(crypto_options[0])  # Set default value

        # Create a text box for messages
        message_label = tk.Label(root, text="Message:")
        message_label.grid(row=1, column=0, padx=10, pady=10, sticky="e")

        self.message_entry = tk.Entry(root, width=40)
        self.message_entry.grid(row=1, column=1, padx=10, pady=10, sticky="w")

        # Create a chat window
        self.chat_window = tk.Text(root, height=15, width=50, state=tk.DISABLED)
        self.chat_window.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

        # Create a send button
        send_button = tk.Button(root, text="Send", command=self.send_message)
        send_button.grid(row=3, column=0, columnspan=2, pady=10)

        # Create a socket
        self.my_socket = socket.socket()
        self.my_socket.bind(("localhost", 4345))
        self.conn = None

        # Start a thread to listen for incoming connections
        threading.Thread(target=self.connector_thread).start()

    def connector_thread(self):
        self.my_socket.listen(5)
        self.conn, addr = self.my_socket.accept()
        print("Device connected")

        threading.Thread(target=self.receive_messages).start()

    def receive_messages(self):
        while True:
            try:
                data = self.conn.recv(1024).decode('utf-8')
                print(data)
                if not data:
                    break

                crypto_technique, message = data.split(":", 1)
                # Perform crypto operations on the received message using the selected technique
                if crypto_technique == "DES":
                    # Add decryption logic here
                    decrypted_message = bin_to_hexa(decrypt(message,key= "0E329232EA6D0D73"))
                    self.display_message(f"{crypto_technique}: {decrypted_message}")
                if crypto_technique == "RC-4":
                    key = "SecretKey"
                    decrypted_message =rc4_decrypt(key, message)
                    self.display_message(f"{crypto_technique}: {decrypted_message}")

            except ConnectionResetError:
                print("Connection closed.")
                break

    def send_message(self):
        if self.conn:
            message = self.message_entry.get()
            crypto_technique = self.crypto_technique_var.get()

            # Perform crypto operations on the message using the selected technique
            if crypto_technique == "DES":
                plain_text=pad(message)
                key= "0E329232EA6D0D73"
                key=pad(key)
                cipher_text=bin_to_hexa(encrypt(plain_text,key))
                self.display_message(f"{crypto_technique}: {cipher_text}")
                

            if crypto_technique == "RC-4":
                key = "SecretKey"
                cipher_text = rc4_encrypt(key, message)
                self.display_message(f"{crypto_technique}: {cipher_text}")

            
            

            # Send the message to the client
            data = f"{crypto_technique}:{cipher_text}"
            msg = bytes(data, 'utf-8')
            self.conn.send(msg)

    def display_message(self, message):
        self.chat_window.config(state=tk.NORMAL)
        self.chat_window.insert(tk.END, message + "\n")
        self.chat_window.config(state=tk.DISABLED)


if __name__ == "__main__":
    root = tk.Tk()
    app = CryptoChatServer(root)
    root.mainloop()

    

Device connected
RC-4:b'v\xd9T&\xbe'
No padding required
No padding required
Encryption
Message after initial permutation:  0000FFFFFF0000FF
The converted 56bit key is:  00010100101100001011101010001001111101100001011100011110

Round: Left key part: Right key part: SubKey used:
01       FF0000FF        A3931787     36146478E1E1
02       A3931787        0DE3C71B     40BD1176E8FD
03       0DE3C71B        4877F40C     45A473239DDB
04       4877F40C        8679354D     E7C4828FB533
05       8679354D        B6908F5F     7A83826F4F64
06       B6908F5F        D33E5656     38901B58C9DE
07       D33E5656        E0E43C4F     25005EC5D49D
08       E0E43C4F        8F274C79     264894CB36E9
09       8F274C79        491DF66E     54554179F633
10       491DF66E        9D8019D7     43C9453F4C2E
11       9D8019D7        89126F65     09E1878C79D6
12       89126F65        25C0DBDA     3105ABA5E2F5
13       25C0DBDA        A94E0CAB     F100A1F38EC3
14       A94E0CAB        507BB97E     918A949E871F
15     